In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory = False)
data.head(2)

In [3]:
#null check
#TF-IDF 연산때 NULL 값이 있으면 에러남
data = data.head(20000)
print(data['overview'].isnull().sum())

135


In [4]:
#null -> empty
data['overview'] = data['overview'].fillna('')

In [7]:
#20000 rows, 47487 columns
#20000 영화를 표현하기 위해 총 47487개의 단어가 사용됨
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF shape:',tfidf_matrix.shape)

TF-IDF shape: (20000, 47487)


In [8]:
#cosine 유사도 계산
#20000개의 문서 벡터간의 유사도 기록됨
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim.shape)

(20000, 20000)


In [9]:
#영화 타이틀을 key, 영화 인덱스를 value로 하는 dictionary만듬
title_to_index = dict(zip(data['title'], data.index))

In [12]:
def get_recommendations(title, cosine_sim = cosine_sim):
  #영화 인덱스 받아오기
  idx = title_to_index[title]

  #해당 영화와 모든 영화와의 유사도 가져옴
  sim_scores = list(enumerate(cosine_sim[idx]))

  #유사도에 따라 영화들 정렬
  sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

  #가장 유력한 5개
  sim_scores = sim_scores[1:6]
  

  #유력한 5개 영화의 인덱스
  movie_indices = [idx[0] for idx in sim_scores]

  return data['title'].iloc[movie_indices]

In [11]:
get_recommendations('The Dark Knight Rises')

12481               The Dark Knight
150                  Batman Forever
1328                 Batman Returns
15511    Batman: Under the Red Hood
585                          Batman
Name: title, dtype: object